In [0]:
import os
import tensorflow as tf
from tensorflow.core.protobuf import saver_pb2
import driving_data
import model
import pandas as pd
import numpy as np

In [0]:
LOGDIR = './save'

In [0]:
df = pd.read_csv("C:\\Users\\K.V\\Desktop\\data.csv", names  = ['imageId', 'angle'], sep = ' ')

In [0]:
df

,imageId,angle
0,0.jpg,0.00
1,1.jpg,0.00
2,2.jpg,0.00
3,3.jpg,0.00
4,4.jpg,0.00
5,5.jpg,0.00
6,6.jpg,0.00
7,7.jpg,0.00
8,8.jpg,0.00
9,9.jpg,0.00


In [0]:
sess = tf.InteractiveSession()

/home/wizard/.local/lib/python3.5/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
L2NormConst = 0.001

In [0]:
train_vars = tf.trainable_variables()

In [0]:
loss = tf.reduce_mean(tf.square(tf.subtract(model.y_, model.y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
sess.run(tf.initialize_all_variables())

create a summary to monitor cost tensor

In [0]:
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op =  tf.summary.merge_all()

In [0]:
saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V1)

op to write logs to Tensorboard

In [0]:
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

In [0]:
epochs = 30
batch_size = 100

In [0]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


train over the dataset about 30 times

In [0]:
for epoch in range(epochs):
  for i in range(int(driving_data.num_images/batch_size)):
    xs, ys = driving_data.LoadTrainBatch(batch_size)
    train_step.run(feed_dict={model.x: xs, model.y_: ys, model.keep_prob: 0.5})
    if i % 10 == 0:
      xs, ys = driving_data.LoadValBatch(batch_size)
      loss_value = loss.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 0.5})
      print("Epoch: %d, Step: %d, Loss: %g" % (epoch, epoch * batch_size + i, loss_value))

    # write logs at every iteration
    summary = merged_summary_op.eval(feed_dict={model.x:xs, model.y_: ys, model.keep_prob: 0.5})
    summary_writer.add_summary(summary, epoch * driving_data.num_images/batch_size + i)
    if i % batch_size == 0:
      if not os.path.exists(LOGDIR):
        os.makedirs(LOGDIR)
      checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
      filename = saver.save(sess, checkpoint_path)
  print("Model saved in file: %s" % filename)

Epoch: 0, Step: 0, Loss: 15.74
Epoch: 0, Step: 10, Loss: 7.2489
Epoch: 0, Step: 20, Loss: 6.47051
Epoch: 0, Step: 30, Loss: 6.39438
Epoch: 0, Step: 40, Loss: 6.09434
Epoch: 0, Step: 50, Loss: 6.20436
Epoch: 0, Step: 60, Loss: 5.89551
Epoch: 0, Step: 70, Loss: 5.85809
Epoch: 0, Step: 80, Loss: 5.86831
Epoch: 0, Step: 90, Loss: 5.74624
Epoch: 0, Step: 100, Loss: 5.71463
Epoch: 0, Step: 110, Loss: 10.4959
Epoch: 0, Step: 120, Loss: 6.34329
Epoch: 0, Step: 130, Loss: 5.53802
Epoch: 0, Step: 140, Loss: 5.48366
Epoch: 0, Step: 150, Loss: 5.43719
Epoch: 0, Step: 160, Loss: 5.39651
Epoch: 0, Step: 170, Loss: 5.35117
Epoch: 0, Step: 180, Loss: 5.32145
Epoch: 0, Step: 190, Loss: 5.26675
Epoch: 0, Step: 200, Loss: 5.24136
Epoch: 0, Step: 210, Loss: 5.19521
Epoch: 0, Step: 220, Loss: 5.15724
Epoch: 0, Step: 230, Loss: 5.13102
Epoch: 0, Step: 240, Loss: 5.1142
Epoch: 0, Step: 250, Loss: 5.06189
Epoch: 0, Step: 260, Loss: 5.02928
Epoch: 0, Step: 270, Loss: 4.98118
Epoch: 0, Step: 280, Loss: 4.94528


Epoch: 2, Step: 310, Loss: 3.35985
Epoch: 2, Step: 320, Loss: 3.15163
Epoch: 2, Step: 330, Loss: 3.13103
Epoch: 2, Step: 340, Loss: 3.11713
Epoch: 2, Step: 350, Loss: 3.10176
Epoch: 2, Step: 360, Loss: 3.08384
Epoch: 2, Step: 370, Loss: 3.07122
Epoch: 2, Step: 380, Loss: 3.05353
Epoch: 2, Step: 390, Loss: 3.03728
Epoch: 2, Step: 400, Loss: 3.02312
Epoch: 2, Step: 410, Loss: 3.00958
Epoch: 2, Step: 420, Loss: 3.0085
Epoch: 2, Step: 430, Loss: 3.0038
Epoch: 2, Step: 440, Loss: 2.97465
Epoch: 2, Step: 450, Loss: 2.9464
Epoch: 2, Step: 460, Loss: 2.93081
Epoch: 2, Step: 470, Loss: 2.91731
Epoch: 2, Step: 480, Loss: 2.9019
Epoch: 2, Step: 490, Loss: 2.88722
Epoch: 2, Step: 500, Loss: 2.87415
Epoch: 2, Step: 510, Loss: 2.8571
Epoch: 2, Step: 520, Loss: 2.84388
Epoch: 2, Step: 530, Loss: 2.83182
Epoch: 2, Step: 540, Loss: 2.81799
Epoch: 2, Step: 550, Loss: 2.80285
Epoch: 2, Step: 560, Loss: 2.78998
Epoch: 2, Step: 570, Loss: 2.77681
Epoch: 2, Step: 580, Loss: 2.76294
Epoch: 2, Step: 590, Loss

Epoch: 4, Step: 610, Loss: 2.01959
Epoch: 4, Step: 620, Loss: 2.01507
Epoch: 4, Step: 630, Loss: 1.99313
Epoch: 4, Step: 640, Loss: 1.97442
Epoch: 4, Step: 650, Loss: 1.96616
Epoch: 4, Step: 660, Loss: 1.95913
Epoch: 4, Step: 670, Loss: 1.95147
Epoch: 4, Step: 680, Loss: 1.94349
Epoch: 4, Step: 690, Loss: 1.93729
Epoch: 4, Step: 700, Loss: 1.92864
Epoch: 4, Step: 710, Loss: 1.92219
Epoch: 4, Step: 720, Loss: 1.91533
Epoch: 4, Step: 730, Loss: 1.90575
Epoch: 4, Step: 740, Loss: 1.89781
Epoch: 4, Step: 750, Loss: 1.89056
Epoch: 4, Step: 760, Loss: 1.88122
Epoch: 4, Step: 770, Loss: 1.87648
Epoch: 4, Step: 780, Loss: 1.87825
Epoch: 4, Step: 790, Loss: 1.92363
Epoch: 4, Step: 800, Loss: 4.55511
Epoch: 4, Step: 810, Loss: 4.6808
Epoch: 4, Step: 820, Loss: 1.84118
Epoch: 4, Step: 830, Loss: 1.83096
Epoch: 4, Step: 840, Loss: 1.8723
Epoch: 4, Step: 850, Loss: 1.8681
Model saved in file: ./save/model.ckpt
Epoch: 5, Step: 500, Loss: 1.8554
Epoch: 5, Step: 510, Loss: 1.85718
Epoch: 5, Step: 520,

Epoch: 6, Step: 910, Loss: 1.36292
Epoch: 6, Step: 920, Loss: 1.35925
Epoch: 6, Step: 930, Loss: 1.35244
Epoch: 6, Step: 940, Loss: 1.34808
Epoch: 6, Step: 950, Loss: 1.34338
Epoch: 6, Step: 960, Loss: 1.34053
Epoch: 6, Step: 970, Loss: 1.3819
Epoch: 6, Step: 980, Loss: 1.41041
Epoch: 6, Step: 990, Loss: 5.4448
Epoch: 6, Step: 1000, Loss: 3.04272
Epoch: 6, Step: 1010, Loss: 1.32695
Epoch: 6, Step: 1020, Loss: 1.31829
Epoch: 6, Step: 1030, Loss: 1.3392
Epoch: 6, Step: 1040, Loss: 1.35228
Epoch: 6, Step: 1050, Loss: 1.3106
Model saved in file: ./save/model.ckpt
Epoch: 7, Step: 700, Loss: 1.39408
Epoch: 7, Step: 710, Loss: 1.382
Epoch: 7, Step: 720, Loss: 1.28682
Epoch: 7, Step: 730, Loss: 1.30907
Epoch: 7, Step: 740, Loss: 1.29579
Epoch: 7, Step: 750, Loss: 1.28915
Epoch: 7, Step: 760, Loss: 1.27293
Epoch: 7, Step: 770, Loss: 1.36291
Epoch: 7, Step: 780, Loss: 1.30164
Epoch: 7, Step: 790, Loss: 1.36946
Epoch: 7, Step: 800, Loss: 1.3277
Epoch: 7, Step: 810, Loss: 1.28139
Epoch: 7, Step: 8

Epoch: 8, Step: 1210, Loss: 0.961799
Epoch: 8, Step: 1220, Loss: 0.994472
Epoch: 8, Step: 1230, Loss: 1.00083
Epoch: 8, Step: 1240, Loss: 0.939819
Epoch: 8, Step: 1250, Loss: 1.03228
Model saved in file: ./save/model.ckpt
Epoch: 9, Step: 900, Loss: 1.01018
Epoch: 9, Step: 910, Loss: 0.925269
Epoch: 9, Step: 920, Loss: 0.964628
Epoch: 9, Step: 930, Loss: 0.945098
Epoch: 9, Step: 940, Loss: 0.945765
Epoch: 9, Step: 950, Loss: 0.921578
Epoch: 9, Step: 960, Loss: 1.00567
Epoch: 9, Step: 970, Loss: 0.926397
Epoch: 9, Step: 980, Loss: 0.968567
Epoch: 9, Step: 990, Loss: 1.02335
Epoch: 9, Step: 1000, Loss: 1.00991
Epoch: 9, Step: 1010, Loss: 0.91389
Epoch: 9, Step: 1020, Loss: 1.0868
Epoch: 9, Step: 1030, Loss: 0.931424
Epoch: 9, Step: 1040, Loss: 1.12542
Epoch: 9, Step: 1050, Loss: 1.03016
Epoch: 9, Step: 1060, Loss: 0.900286
Epoch: 9, Step: 1070, Loss: 0.891528
Epoch: 9, Step: 1080, Loss: 0.884057
Epoch: 9, Step: 1090, Loss: 0.875955
Epoch: 9, Step: 1100, Loss: 0.87053
Epoch: 9, Step: 1110,

...

Epoch: 19, Step: 2210, Loss: 0.212063
Epoch: 19, Step: 2220, Loss: 0.22445
Epoch: 19, Step: 2230, Loss: 0.266518
Epoch: 19, Step: 2240, Loss: 0.350511
Epoch: 19, Step: 2250, Loss: 0.241714
Epoch: 19, Step: 2260, Loss: 1.01817
Epoch: 19, Step: 2270, Loss: 0.762453
Epoch: 19, Step: 2280, Loss: 0.284847
Epoch: 19, Step: 2290, Loss: 0.450757
Epoch: 19, Step: 2300, Loss: 0.225669
Epoch: 19, Step: 2310, Loss: 0.2211
Epoch: 19, Step: 2320, Loss: 0.234026
Epoch: 19, Step: 2330, Loss: 0.249866
Epoch: 19, Step: 2340, Loss: 0.237543
Epoch: 19, Step: 2350, Loss: 4.21785
Model saved in file: ./save/model.ckpt
Epoch: 20, Step: 2000, Loss: 1.60351
Epoch: 20, Step: 2010, Loss: 0.209747
Epoch: 20, Step: 2020, Loss: 0.20344
Epoch: 20, Step: 2030, Loss: 0.203563
Epoch: 20, Step: 2040, Loss: 0.216715
Epoch: 20, Step: 2050, Loss: 0.202409
Epoch: 20, Step: 2060, Loss: 0.243214
Epoch: 20, Step: 2070, Loss: 0.212609
Epoch: 20, Step: 2080, Loss: 0.201516
Epoch: 20, Step: 2090, Loss: 0.207493
Epoch: 20, Step: 2

Epoch: 21, Step: 2510, Loss: 0.258263
Epoch: 21, Step: 2520, Loss: 0.269549
Epoch: 21, Step: 2530, Loss: 0.400253
Epoch: 21, Step: 2540, Loss: 4.97286
Epoch: 21, Step: 2550, Loss: 0.383623
Model saved in file: ./save/model.ckpt
Epoch: 22, Step: 2200, Loss: 0.183576
Epoch: 22, Step: 2210, Loss: 0.182316
Epoch: 22, Step: 2220, Loss: 0.232716
Epoch: 22, Step: 2230, Loss: 0.215344
Epoch: 22, Step: 2240, Loss: 0.40373
Epoch: 22, Step: 2250, Loss: 0.561418
Epoch: 22, Step: 2260, Loss: 1.36685
Epoch: 22, Step: 2270, Loss: 0.808611
Epoch: 22, Step: 2280, Loss: 0.795542
Epoch: 22, Step: 2290, Loss: 0.971607
Epoch: 22, Step: 2300, Loss: 0.694063
Epoch: 22, Step: 2310, Loss: 0.21833
Epoch: 22, Step: 2320, Loss: 0.322695
Epoch: 22, Step: 2330, Loss: 0.182358
Epoch: 22, Step: 2340, Loss: 0.173945
Epoch: 22, Step: 2350, Loss: 0.173644
Epoch: 22, Step: 2360, Loss: 0.179332
Epoch: 22, Step: 2370, Loss: 0.173229
Epoch: 22, Step: 2380, Loss: 0.172983
Epoch: 22, Step: 2390, Loss: 0.17194
Epoch: 22, Step:

Epoch: 24, Step: 2400, Loss: 0.167007
Epoch: 24, Step: 2410, Loss: 0.174235
Epoch: 24, Step: 2420, Loss: 0.179633
Epoch: 24, Step: 2430, Loss: 0.168382
Epoch: 24, Step: 2440, Loss: 0.184754
Epoch: 24, Step: 2450, Loss: 0.167156
Epoch: 24, Step: 2460, Loss: 0.172697
Epoch: 24, Step: 2470, Loss: 0.18201
Epoch: 24, Step: 2480, Loss: 0.306279
Epoch: 24, Step: 2490, Loss: 0.17789
Epoch: 24, Step: 2500, Loss: 0.181161
Epoch: 24, Step: 2510, Loss: 0.2318
Epoch: 24, Step: 2520, Loss: 0.167434
Epoch: 24, Step: 2530, Loss: 0.171891
Epoch: 24, Step: 2540, Loss: 0.231097
Epoch: 24, Step: 2550, Loss: 0.179642
Epoch: 24, Step: 2560, Loss: 0.162546
Epoch: 24, Step: 2570, Loss: 0.173809
Epoch: 24, Step: 2580, Loss: 0.176634
Epoch: 24, Step: 2590, Loss: 0.184918
Epoch: 24, Step: 2600, Loss: 0.182155
Epoch: 24, Step: 2610, Loss: 0.203044
Epoch: 24, Step: 2620, Loss: 0.171621
Epoch: 24, Step: 2630, Loss: 0.191407
Epoch: 24, Step: 2640, Loss: 0.240366
Epoch: 24, Step: 2650, Loss: 0.185835
Epoch: 24, Step:

Epoch: 26, Step: 2640, Loss: 0.201877
Epoch: 26, Step: 2650, Loss: 0.429103
Epoch: 26, Step: 2660, Loss: 0.49921
Epoch: 26, Step: 2670, Loss: 1.77337
Epoch: 26, Step: 2680, Loss: 0.309628
Epoch: 26, Step: 2690, Loss: 0.290203
Epoch: 26, Step: 2700, Loss: 0.199128
Epoch: 26, Step: 2710, Loss: 0.249781
Epoch: 26, Step: 2720, Loss: 0.154076
Epoch: 26, Step: 2730, Loss: 0.149006
Epoch: 26, Step: 2740, Loss: 0.19011
Epoch: 26, Step: 2750, Loss: 0.149417
Epoch: 26, Step: 2760, Loss: 0.157085
Epoch: 26, Step: 2770, Loss: 0.149493
Epoch: 26, Step: 2780, Loss: 0.151174
Epoch: 26, Step: 2790, Loss: 0.149716
Epoch: 26, Step: 2800, Loss: 0.148226
Epoch: 26, Step: 2810, Loss: 0.149236
Epoch: 26, Step: 2820, Loss: 0.149778
Epoch: 26, Step: 2830, Loss: 0.146988
Epoch: 26, Step: 2840, Loss: 0.150356
Epoch: 26, Step: 2850, Loss: 0.171659
Epoch: 26, Step: 2860, Loss: 0.193069
Epoch: 26, Step: 2870, Loss: 3.31225
Epoch: 26, Step: 2880, Loss: 2.37639
Epoch: 26, Step: 2890, Loss: 0.155413
Epoch: 26, Step: 

Epoch: 28, Step: 2910, Loss: 0.141136
Epoch: 28, Step: 2920, Loss: 0.15163
Epoch: 28, Step: 2930, Loss: 0.156274
Epoch: 28, Step: 2940, Loss: 0.14658
Epoch: 28, Step: 2950, Loss: 0.180456
Epoch: 28, Step: 2960, Loss: 0.170442
Epoch: 28, Step: 2970, Loss: 0.207677
Epoch: 28, Step: 2980, Loss: 0.146276
Epoch: 28, Step: 2990, Loss: 0.159429
Epoch: 28, Step: 3000, Loss: 0.148291
Epoch: 28, Step: 3010, Loss: 0.206128
Epoch: 28, Step: 3020, Loss: 0.244975
Epoch: 28, Step: 3030, Loss: 0.158432
Epoch: 28, Step: 3040, Loss: 0.527541
Epoch: 28, Step: 3050, Loss: 0.316938
Epoch: 28, Step: 3060, Loss: 5.54087
Epoch: 28, Step: 3070, Loss: 1.27369
Epoch: 28, Step: 3080, Loss: 0.244231
Epoch: 28, Step: 3090, Loss: 0.357366
Epoch: 28, Step: 3100, Loss: 0.465714
Epoch: 28, Step: 3110, Loss: 0.189653
Epoch: 28, Step: 3120, Loss: 0.149936
Epoch: 28, Step: 3130, Loss: 0.185684
Epoch: 28, Step: 3140, Loss: 0.196658
Epoch: 28, Step: 3150, Loss: 0.14127
Epoch: 28, Step: 3160, Loss: 0.176174
Epoch: 28, Step: 

In [0]:
print("Run the command line:\n" \
          "--> tensorboard --logdir=./logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser")